## The Hamiltonian for the spring-ball oscillating system given in paper https://arxiv.org/pdf/2303.13012.pdf

Following largely from section 3 of the paper

- $ H = -\begin{bmatrix} 0 & B \\ B^\dagger & 0 \end{bmatrix} $ where dim of $B$ is $N \text{x} M$, therefore $dim(H) = (N+M) \text{x} (N+M)$
- We pad $B$ with zeros such that $dim (B) = N^2 \text{x}N^2$ (according to appendix A1 and A4), therefore $dim(H) = 2N^2 \text{x} 2N^2$
- Now for $B$ we have, $BB^\dagger = A$, $ \begin{equation}
  \sqrt{M}B|j,k\rangle =
    \begin{cases}
      \sqrt{k_{jj}}|j\rangle & \text{if } j=k \\
      \sqrt{k_{jk}}(|j\rangle-|k\rangle) & \text{if } j<k\\
    \end{cases}       
\end{equation} $ and elements of $B^\dagger$ are either $\sqrt{k_{jk}/m_j}$ or $0$.

- $ A$ = $\sqrt{M}^{-1] F \sqrt{M}^{-1} $.

- $M$ is diagonal matrix of masses $(m_{jj}>0)$ and $F$ is the $N \text{x} N$ matrix whose diagonal and off-diagonal entries are $f_{jj} = \sum_k \kappa_{jk}$ and $f_{jk} = -\kappa_{jk}$,respectively. 

for the second point in $|j,k\rangle$ state is in vector [00,11,22,33,01,02,03,12,13,23]

#### Case : $$d = 1, E = 1, n = 2 \implies N = 4, \text{ no. of springs = 4 }, m_i = 1 \forall i \in [1,N], k_{i,j} \in [1,4], j>i, \forall (i,j) $$
$$\dot{\vec{x}}(0) = \begin{bmatrix} 1 \\ -1 \\ 0 \\  0 \end{bmatrix} \text{ and } \vec{x} = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$ 

In [17]:
import numpy as np
import scipy as sc
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import HGate
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from utils import Hamiltonian_Formulation
from classiq import (
    Output,
    QArray,
    QParam,
    List,
    QBit,
    QNum,
    control,
    create_model,
    execute,
    prepare_state,
    qfunc,
    show,
    synthesize,
    H,
    Output,
    Z,
    allocate,
    qfunc,
    suzuki_trotter,
)

from classiq.applications.chemistry import (
    ChemistryExecutionParameters,
    HEAParameters,
    Molecule,
    MoleculeProblem,
    UCCParameters,
    FermionicOperator,
    HamiltonianProblem,
)

import classiq
from qiskit import qasm3

# classiq.authenticate()

In [2]:
n = 2
N = 2**n
m = 1
E = 1
#mass matrix
M = np.diag(np.full(N,m)) 
#K matrix (spring constants)
k11 = 2
k12 = 1
k23 = 3
k34 = 2
K = np.array(([k11,k12,0,0],[k12,0,k23,0],[0,k23,0,k34],[0,0,k34,0]))


ham_formulation = Hamiltonian_Formulation(n, K, M)
ham_formulation.compute_Hamiltonian()
H_matrix = ham_formulation.H_matrix
H_matrix[1]
# print(len(H_matrix))

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.        , -0.        , -0.        , -0.        ,
          1.        , -0.        , -0.        , -1.73205081, -0.        ,
         -0.        , -0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ]])

## State Preparation

In [3]:
initial_state = np.array([1/np.sqrt(2),-1/np.sqrt(2)] + [0 for i in range(2*N**2-2)])
print(len(initial_state))
# use https://github.com/Qiskit/qiskit/issues/11735#issuecomment-1992145075 or Classiq

qc = QuantumCircuit(2*n+1)
qc.h(0)
qc.z(0)

init_state = Statevector.from_instruction(qc)

initial_state


32


array([ 0.70710678, -0.70710678,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

## Conditions for testing
- t = 5
- for n = 2, 5, 10
- accuracy = 99%
- note down accuracy, depth of circuit, number of gates (single and 2 qubit)

## Computing the final state classically

In [4]:
t=5

# exact_times = np.linspace(0, t, 101)
# We define a slightly denser time mesh
exact_times = np.linspace(0, t, 101)

# We compute the exact evolution using the exp
final_state = init_state.evolve(sc.linalg.expm(-1j * t * H_matrix)) 

In [5]:
final_state.is_valid()

True

## Computing the Hamiltonian simulation on Classiq

### State preparation using Classiq

In [14]:
H_sparse_pauli.to_list()

[('XIIII', (-0.08838834764831845+0j)),
 ('XIIIZ', (-0.08838834764831845+0j)),
 ('XIIZI', (-0.08838834764831845+0j)),
 ('XIIZZ', (-0.08838834764831845+0j)),
 ('XIXII', (-0.0625+0j)),
 ('XIXIX', (0.17075317547305482+0j)),
 ('XIXIZ', (-0.0625+0j)),
 ('XIXXI', (-0.10825317547305482+0j)),
 ('XIXXZ', (0.10825317547305482+0j)),
 ('XIXZI', (-0.0625+0j)),
 ('XIXZX', (-0.045753175473054825+0j)),
 ('XIXZZ', (-0.0625+0j)),
 ('XIYIY', (-0.045753175473054825+0j)),
 ('XIYYI', (0.10825317547305482-0j)),
 ('XIYYZ', (-0.10825317547305482+0j)),
 ('XIYZY', (0.17075317547305482-0j)),
 ('XIZII', (-0.08838834764831845+0j)),
 ('XIZIZ', (-0.08838834764831845+0j)),
 ('XIZZI', (-0.08838834764831845+0j)),
 ('XIZZZ', (-0.08838834764831845+0j)),
 ('XXIXI', (0.08838834764831845+0j)),
 ('XXIXX', (-0.08838834764831845+0j)),
 ('XXIXZ', (-0.08838834764831845+0j)),
 ('XXIYY', (-0.08838834764831845+0j)),
 ('XXZXI', (0.08838834764831845+0j)),
 ('XXZXX', (-0.08838834764831845+0j)),
 ('XXZXZ', (-0.08838834764831845+0j)),
 ('

In [26]:
# creating the Sparse Pauli Op
H_sparse_pauli = SparsePauliOp.from_operator(H_matrix)

# State preparation in Classiq
# importing the qmod file
@qfunc
def state_prep(unchanged: Output[QArray[QBit]]):
    allocate(2*n+1, unchanged)
    H(unchanged[0])
    Z(unchanged[0])
    
@qfunc
def main(io: Output[QArray[QBit]]) -> None:
    state = prepare_state(probabilities=(initial_state**2).tolist(), bound=0.01, out=io)
    Z(io[0])

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/fd13640e-2bd7-4898-a505-125248fca81c?version=0.39.0
